In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import sqrt
from tensorflow import keras
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *


from preprocess import water_process
# from model.cnn import cnn_4layer, mlp_layer
from model.mlp import mlp_layer

2024-01-30 15:59:23.101890: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
np.percentile(data['MEAN_RAIN'], 95)

0.0053125

### Preprocess

In [5]:
# ====== preprocessing parameters ======
n_hours = 72
k = 24 
masked_value = 1e-10
split_1 = 0.7
split_2 = 0.85

In [6]:
train_X_mask, val_X_mask, test_X_mask, \
train_ws_y, val_ws_y, test_ws_y, \
scaler, ws_scaler = water_process(n_hours, k, masked_value, split_1, split_2)

Index(['MEAN_RAIN', 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2',
       'GATE_S26_1', 'GATE_S26_2', 'PUMP_S25B', 'PUMP_S26', 'HWS_S25A',
       'HWS_S25B', 'HWS_S26', 'WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26'],
      dtype='object')


In [7]:
print(train_X_mask.shape, val_X_mask.shape, test_X_mask.shape, train_ws_y.shape, val_ws_y.shape, test_ws_y.shape)

(67435, 96, 16) (14451, 96, 16) (14451, 96, 16) (67435, 96) (14451, 96) (14451, 96)


### Rain threshold

In [8]:
# train_X_mask_rain_acc = np.sum(train_X_mask[:, -24:, 0], axis=1)
# print(train_X_mask_rain_acc.shape)

# test_X_mask_rain_acc = np.sum(test_X_mask[:, -24:, 0], axis=1)
# print(test_X_mask_rain_acc.shape)

train_X_mask_rain_acc = train_X_mask[:, 0, 0]
print(train_X_mask_rain_acc.shape)

val_X_mask_rain_acc = val_X_mask[:, 0, 0]
print(val_X_mask_rain_acc.shape)

test_X_mask_rain_acc = test_X_mask[:, 0, 0]
print(test_X_mask_rain_acc.shape)

(67435,)
(14451,)
(14451,)


In [9]:
percentile = 95

merged_array = np.concatenate((train_X_mask_rain_acc, val_X_mask_rain_acc, test_X_mask_rain_acc))

percentile_rain = np.percentile(merged_array, percentile)

print("{}th Percentile of Daily Rain:".format(percentile), percentile_rain)

95th Percentile of Daily Rain: 0.010711610503493287


### train_X_mask_filter

In [10]:
# ======= extreme =======
train_X_mask_extreme = train_X_mask[train_X_mask_rain_acc > percentile_rain]
print(train_X_mask_extreme.shape)

train_ws_y_extreme = train_ws_y[train_X_mask_rain_acc > percentile_rain]
print(train_ws_y_extreme.shape)


# ======= normal =======
train_X_mask_normal = train_X_mask[train_X_mask_rain_acc <= percentile_rain]
print(train_X_mask_normal.shape)

train_ws_y_normal = train_ws_y[train_X_mask_rain_acc <= percentile_rain]
print(train_ws_y_normal.shape)

(3251, 96, 16)
(3251, 96)
(64184, 96, 16)
(64184, 96)


### val_X_mask_filter

In [11]:
# ======= extreme =======
val_X_mask_extreme = val_X_mask[val_X_mask_rain_acc > percentile_rain]
print(val_X_mask_extreme.shape)

val_ws_y_extreme = val_ws_y[val_X_mask_rain_acc > percentile_rain]
print(val_ws_y_extreme.shape)


# ======= normal =======
val_X_mask_normal = val_X_mask[val_X_mask_rain_acc <= percentile_rain]
print(val_X_mask_normal.shape)

val_ws_y_normal = val_ws_y[val_X_mask_rain_acc <= percentile_rain]
print(val_ws_y_normal.shape)

(536, 96, 16)
(536, 96)
(13915, 96, 16)
(13915, 96)


### test_X_mask_filter

In [12]:
# ======= extreme =======
test_X_mask_extreme = test_X_mask[test_X_mask_rain_acc > percentile_rain]
print(test_X_mask_extreme.shape)

test_ws_y_extreme = test_ws_y[test_X_mask_rain_acc > percentile_rain]
print(test_ws_y_extreme.shape)


# ======= normal =======
test_X_mask_normal = test_X_mask[test_X_mask_rain_acc <= percentile_rain]
print(test_X_mask_normal.shape)

test_ws_y_normal = test_ws_y[test_X_mask_rain_acc <= percentile_rain]
print(test_ws_y_normal.shape)

(1030, 96, 16)
(1030, 96)
(13421, 96, 16)
(13421, 96)


### Model

In [13]:
# ===== model parameters ======
mlp_unit1 = 128
mlp_unit2 = 128
mlp_unit3 = 64
mlp_unit4 = 64
mlp_unit5 = 32
mlp_unit6 = 32
mlp_unit7 = 16
mlp_unit8 = 16
dropout = 0.0  # 0.1
kernel_size = 2
pool_size = 2
learning_rate = 1e-4
decay_steps = 10000
decay_rate = 0.95
PATIENCE = 100
EPOCHS = 1000
BATCH = 512
opt_num = k*4
input_shape = train_X_mask.shape[1:]

In [15]:
model = mlp_layer(input_shape=input_shape,
                   mlp_unit1=mlp_unit1,
                   mlp_unit2=mlp_unit2,
                   mlp_unit3=mlp_unit3,
                   mlp_unit4=mlp_unit4,
                   mlp_unit5=mlp_unit5,
                   mlp_unit6=mlp_unit6,
                   mlp_unit7=mlp_unit7,
                   mlp_unit8=mlp_unit8,
                   dropout=dropout,
                   masked_value=-1,
                   opt_num=opt_num
                  )
# model.summary()

In [16]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate, 
                                                          decay_steps=decay_steps,
                                                          decay_rate=decay_rate)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='mse',
              metrics=['mae']
             )


es = EarlyStopping(monitor='val_mae', mode='min', verbose=2, patience=PATIENCE)
mc = ModelCheckpoint('../../saved_models/water_E_E_95.h5', 
                     monitor='val_mae', 
                     mode='min', 
                     verbose=2, 
                     save_best_only=True
                    )


model.fit([train_X_mask_extreme], train_ws_y_extreme,
          validation_data=([val_X_mask_extreme], val_ws_y_extreme),
          epochs=EPOCHS,
          batch_size=BATCH,
          verbose=2,
          shuffle=True,
          callbacks=[es, mc]
         )

2024-01-30 15:59:56.870807: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-30 15:59:56.871812: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1700105000 Hz


Epoch 1/1000


2024-01-30 15:59:58.784366: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


7/7 - 5s - loss: 0.1559 - mae: 0.3722 - val_loss: 0.2477 - val_mae: 0.4675

Epoch 00001: val_mae improved from inf to 0.46752, saving model to ../../saved_models/water_E_E_95.h5
Epoch 2/1000
7/7 - 0s - loss: 0.1403 - mae: 0.3502 - val_loss: 0.2191 - val_mae: 0.4354

Epoch 00002: val_mae improved from 0.46752 to 0.43545, saving model to ../../saved_models/water_E_E_95.h5
Epoch 3/1000
7/7 - 0s - loss: 0.1152 - mae: 0.3113 - val_loss: 0.1733 - val_mae: 0.3784

Epoch 00003: val_mae improved from 0.43545 to 0.37836, saving model to ../../saved_models/water_E_E_95.h5
Epoch 4/1000
7/7 - 0s - loss: 0.0802 - mae: 0.2487 - val_loss: 0.1122 - val_mae: 0.2893

Epoch 00004: val_mae improved from 0.37836 to 0.28931, saving model to ../../saved_models/water_E_E_95.h5
Epoch 5/1000
7/7 - 0s - loss: 0.0439 - mae: 0.1726 - val_loss: 0.0560 - val_mae: 0.1926

Epoch 00005: val_mae improved from 0.28931 to 0.19261, saving model to ../../saved_models/water_E_E_95.h5
Epoch 6/1000
7/7 - 0s - loss: 0.0252 - mae

7/7 - 0s - loss: 0.0018 - mae: 0.0320 - val_loss: 0.0040 - val_mae: 0.0501

Epoch 00045: val_mae improved from 0.05012 to 0.05011, saving model to ../../saved_models/water_E_E_95.h5
Epoch 46/1000
7/7 - 0s - loss: 0.0018 - mae: 0.0318 - val_loss: 0.0039 - val_mae: 0.0493

Epoch 00046: val_mae improved from 0.05011 to 0.04926, saving model to ../../saved_models/water_E_E_95.h5
Epoch 47/1000
7/7 - 0s - loss: 0.0017 - mae: 0.0315 - val_loss: 0.0039 - val_mae: 0.0493

Epoch 00047: val_mae did not improve from 0.04926
Epoch 48/1000
7/7 - 0s - loss: 0.0017 - mae: 0.0313 - val_loss: 0.0038 - val_mae: 0.0487

Epoch 00048: val_mae improved from 0.04926 to 0.04866, saving model to ../../saved_models/water_E_E_95.h5
Epoch 49/1000
7/7 - 0s - loss: 0.0017 - mae: 0.0312 - val_loss: 0.0037 - val_mae: 0.0485

Epoch 00049: val_mae improved from 0.04866 to 0.04848, saving model to ../../saved_models/water_E_E_95.h5
Epoch 50/1000
7/7 - 0s - loss: 0.0017 - mae: 0.0310 - val_loss: 0.0036 - val_mae: 0.0479



...


Epoch 00744: val_mae did not improve from 0.02874
Epoch 745/1000
7/7 - 0s - loss: 1.8372e-04 - mae: 0.0088 - val_loss: 0.0014 - val_mae: 0.0303

Epoch 00745: val_mae did not improve from 0.02874
Epoch 746/1000
7/7 - 0s - loss: 1.8268e-04 - mae: 0.0087 - val_loss: 0.0013 - val_mae: 0.0291

Epoch 00746: val_mae did not improve from 0.02874
Epoch 747/1000
7/7 - 0s - loss: 1.8248e-04 - mae: 0.0087 - val_loss: 0.0013 - val_mae: 0.0299

Epoch 00747: val_mae did not improve from 0.02874
Epoch 748/1000
7/7 - 0s - loss: 1.8236e-04 - mae: 0.0087 - val_loss: 0.0014 - val_mae: 0.0301

Epoch 00748: val_mae did not improve from 0.02874
Epoch 749/1000
7/7 - 0s - loss: 1.8222e-04 - mae: 0.0087 - val_loss: 0.0013 - val_mae: 0.0293

Epoch 00749: val_mae did not improve from 0.02874
Epoch 750/1000
7/7 - 0s - loss: 1.8008e-04 - mae: 0.0086 - val_loss: 0.0013 - val_mae: 0.0295

Epoch 00750: val_mae did not improve from 0.02874
Epoch 751/1000
7/7 - 0s - loss: 1.7989e-04 - mae: 0.0086 - val_loss: 0.0013 - v

7/7 - 0s - loss: 1.7503e-04 - mae: 0.0086 - val_loss: 0.0013 - val_mae: 0.0289

Epoch 00800: val_mae did not improve from 0.02852
Epoch 801/1000
7/7 - 0s - loss: 1.7382e-04 - mae: 0.0085 - val_loss: 0.0013 - val_mae: 0.0294

Epoch 00801: val_mae did not improve from 0.02852
Epoch 802/1000
7/7 - 0s - loss: 1.7378e-04 - mae: 0.0085 - val_loss: 0.0013 - val_mae: 0.0287

Epoch 00802: val_mae did not improve from 0.02852
Epoch 803/1000
7/7 - 0s - loss: 1.7367e-04 - mae: 0.0085 - val_loss: 0.0013 - val_mae: 0.0296

Epoch 00803: val_mae did not improve from 0.02852
Epoch 804/1000
7/7 - 0s - loss: 1.7621e-04 - mae: 0.0086 - val_loss: 0.0012 - val_mae: 0.0285

Epoch 00804: val_mae improved from 0.02852 to 0.02851, saving model to ../../saved_models/water_E_E_95.h5
Epoch 805/1000
7/7 - 0s - loss: 1.7373e-04 - mae: 0.0085 - val_loss: 0.0013 - val_mae: 0.0295

Epoch 00805: val_mae did not improve from 0.02851
Epoch 806/1000
7/7 - 0s - loss: 1.7500e-04 - mae: 0.0086 - val_loss: 0.0012 - val_mae: 0.


Epoch 00855: val_mae did not improve from 0.02812
Epoch 856/1000
7/7 - 0s - loss: 1.6846e-04 - mae: 0.0084 - val_loss: 0.0012 - val_mae: 0.0286

Epoch 00856: val_mae did not improve from 0.02812
Epoch 857/1000
7/7 - 0s - loss: 1.6727e-04 - mae: 0.0083 - val_loss: 0.0013 - val_mae: 0.0290

Epoch 00857: val_mae did not improve from 0.02812
Epoch 858/1000
7/7 - 0s - loss: 1.6727e-04 - mae: 0.0084 - val_loss: 0.0013 - val_mae: 0.0297

Epoch 00858: val_mae did not improve from 0.02812
Epoch 859/1000
7/7 - 0s - loss: 1.6823e-04 - mae: 0.0084 - val_loss: 0.0013 - val_mae: 0.0289

Epoch 00859: val_mae did not improve from 0.02812
Epoch 860/1000
7/7 - 0s - loss: 1.6659e-04 - mae: 0.0083 - val_loss: 0.0013 - val_mae: 0.0290

Epoch 00860: val_mae did not improve from 0.02812
Epoch 861/1000
7/7 - 0s - loss: 1.6676e-04 - mae: 0.0083 - val_loss: 0.0013 - val_mae: 0.0291

Epoch 00861: val_mae did not improve from 0.02812
Epoch 862/1000
7/7 - 0s - loss: 1.6692e-04 - mae: 0.0083 - val_loss: 0.0013 - v


Epoch 00911: val_mae did not improve from 0.02800
Epoch 912/1000
7/7 - 0s - loss: 1.6281e-04 - mae: 0.0083 - val_loss: 0.0013 - val_mae: 0.0288

Epoch 00912: val_mae did not improve from 0.02800
Epoch 913/1000
7/7 - 0s - loss: 1.6209e-04 - mae: 0.0083 - val_loss: 0.0012 - val_mae: 0.0283

Epoch 00913: val_mae did not improve from 0.02800
Epoch 914/1000
7/7 - 0s - loss: 1.6135e-04 - mae: 0.0082 - val_loss: 0.0012 - val_mae: 0.0284

Epoch 00914: val_mae did not improve from 0.02800
Epoch 915/1000
7/7 - 0s - loss: 1.6180e-04 - mae: 0.0082 - val_loss: 0.0013 - val_mae: 0.0293

Epoch 00915: val_mae did not improve from 0.02800
Epoch 916/1000
7/7 - 0s - loss: 1.6191e-04 - mae: 0.0082 - val_loss: 0.0013 - val_mae: 0.0294

Epoch 00916: val_mae did not improve from 0.02800
Epoch 917/1000
7/7 - 0s - loss: 1.6232e-04 - mae: 0.0083 - val_loss: 0.0012 - val_mae: 0.0281

Epoch 00917: val_mae did not improve from 0.02800
Epoch 918/1000
7/7 - 0s - loss: 1.6734e-04 - mae: 0.0085 - val_loss: 0.0013 - v

7/7 - 0s - loss: 1.6071e-04 - mae: 0.0083 - val_loss: 0.0013 - val_mae: 0.0299

Epoch 00967: val_mae did not improve from 0.02746
Epoch 968/1000
7/7 - 0s - loss: 1.6105e-04 - mae: 0.0083 - val_loss: 0.0012 - val_mae: 0.0277

Epoch 00968: val_mae did not improve from 0.02746
Epoch 969/1000
7/7 - 0s - loss: 1.5689e-04 - mae: 0.0081 - val_loss: 0.0013 - val_mae: 0.0289

Epoch 00969: val_mae did not improve from 0.02746
Epoch 970/1000
7/7 - 0s - loss: 1.5732e-04 - mae: 0.0082 - val_loss: 0.0012 - val_mae: 0.0281

Epoch 00970: val_mae did not improve from 0.02746
Epoch 971/1000
7/7 - 0s - loss: 1.5531e-04 - mae: 0.0081 - val_loss: 0.0012 - val_mae: 0.0286

Epoch 00971: val_mae did not improve from 0.02746
Epoch 972/1000
7/7 - 0s - loss: 1.5813e-04 - mae: 0.0082 - val_loss: 0.0013 - val_mae: 0.0287

Epoch 00972: val_mae did not improve from 0.02746
Epoch 973/1000
7/7 - 0s - loss: 1.5952e-04 - mae: 0.0082 - val_loss: 0.0012 - val_mae: 0.0277

Epoch 00973: val_mae did not improve from 0.02746
